# Analisando o bot

Este jupyter-notebook vai auxiliar na análise de detecção de intenções da Taís.

## Instalação

### Configurando jupyter

In [1]:
%matplotlib inline

import logging, io, json, warnings
logging.basicConfig(level="INFO")
warnings.filterwarnings('ignore')

def pprint(o):
    # small helper to make dict dumps a bit prettier
    print(json.dumps(o, indent=2))

### Listando versões

In [2]:
import rasa_nlu

print("rasa_nlu: {}".format(rasa_nlu.__version__))

rasa_nlu: 0.14.1


### Treinando o modelo do Rasa NLU

Na celula abaixo todas as intents na pasta `../data/intents/` serão utilizadas para gerar a matrix de confuzão para de avaliação do bot.

Ela irá treinar o modelo e executar a avaliação que pode ser verificada na saída da célula abaixo.

In [3]:
import os

from rasa_nlu.training_data import load_data
from rasa_nlu.model import Trainer
from rasa_nlu import config


intents_directory = '../../bot/data/intents/'

intents = {}

for intent_file in os.listdir(intents_directory):
    intent_file_path = os.path.join(intents_directory, intent_file)

    intents[intent_file] = {}

    intent_list = []
    intent_name = None

    with open(intent_file_path) as f:
        lines = f.readlines()

        for line in lines:
            line = line.strip()

            if line.startswith('##'):
                if intent_name is not None:
                    intents[intent_file][intent_name] = intent_list
                intent_name = line.replace('## intent:', '') 
                intent_list = []

            elif line.startswith('- '):
                intent_list.append(line.replace('- ', ''))


# loading the nlu training samples
training_data = load_data(intents_directory)

# trainer to educate our pipeline
trainer = Trainer(config.load("../../bot/nlu_config.yml"))

# train the model!
interpreter = trainer.train(training_data)

# store it for future use
model_directory = trainer.persist("./models/nlu", fixed_model_name="current")

INFO:rasa_nlu.training_data.loading:Training data format of ../../bot/data/intents/civel.md is md
INFO:rasa_nlu.training_data.training_data:Training data stats: 
	- intent examples: 85 (12 distinct intents)
	- Found intents: 'civel_defesa', 'civel_defesa_localprocesso', 'civel_propositura', 'menu_lembrete_podepagaradvogado', 'civel', 'menu_lembrete_naopodepagaradvogado', 'menu_lembrete', 'civel_propositura_pessoafisica_localmoradia', 'civel_propositura_pessoafisica', 'civel_defesa_digamais', 'civel_propositura_pessoafisica_cejusc', 'civel_propositura_pessoajuridica'
	- entity examples: 0 (0 distinct entities)
	- found entities: 

INFO:rasa_nlu.training_data.loading:Training data format of ../../bot/data/intents/actions.md is md
INFO:rasa_nlu.training_data.training_data:Training data stats: 
	- intent examples: 24 (2 distinct intents)
	- Found intents: 'action_confirma_agendamento', 'cejusc'
	- entity examples: 19 (1 distinct entities)
	- found entities: 'local'

INFO:rasa_nlu.training_

In [4]:
%rm errors.json
from rasa_nlu.evaluate import run_evaluation
run_evaluation('../../bot/data/intents/', model_directory)

rm: cannot remove 'errors.json': No such file or directory


INFO:tensorflow:Restoring parameters from /work/notebooks/intents/./models/nlu/default/current/intent_classifier_tensorflow_embedding.ckpt
INFO:rasa_nlu.training_data.loading:Training data format of ../../bot/data/intents/civel.md is md
INFO:rasa_nlu.training_data.training_data:Training data stats: 
	- intent examples: 85 (12 distinct intents)
	- Found intents: 'civel_defesa', 'civel_defesa_localprocesso', 'civel_propositura', 'menu_lembrete_podepagaradvogado', 'civel', 'menu_lembrete_naopodepagaradvogado', 'menu_lembrete', 'civel_propositura_pessoafisica_localmoradia', 'civel_propositura_pessoafisica', 'civel_defesa_digamais', 'civel_propositura_pessoafisica_cejusc', 'civel_propositura_pessoajuridica'
	- entity examples: 0 (0 distinct entities)
	- found entities: 

INFO:rasa_nlu.training_data.loading:Training data format of ../../bot/data/intents/actions.md is md
INFO:rasa_nlu.training_data.training_data:Training data stats: 
	- intent examples: 24 (2 distinct intents)
	- Found intent

{'intent_evaluation': {'predictions': [{'text': 'cível',
    'intent': 'civel',
    'predicted': 'civel',
    'confidence': 0.9329460263252258},
   {'text': 'civel',
    'intent': 'civel',
    'predicted': 'civel',
    'confidence': 0.9391365051269531},
   {'text': 'Cível e Família',
    'intent': 'civel',
    'predicted': 'civel',
    'confidence': 0.9527733325958252},
   {'text': 'civel e familia',
    'intent': 'civel',
    'predicted': 'civel',
    'confidence': 0.9543226957321167},
   {'text': 'Área Cível e Família',
    'intent': 'civel',
    'predicted': 'civel',
    'confidence': 0.948814332485199},
   {'text': 'preciso de encaminhamento para cível e família',
    'intent': 'civel',
    'predicted': 'civel',
    'confidence': 0.9622155427932739},
   {'text': 'atendimento em cível',
    'intent': 'civel',
    'predicted': 'civel',
    'confidence': 0.9446220993995667},
   {'text': 'Eu posso ser defendido pela Defensoria',
    'intent': 'menu_lembrete',
    'predicted': 'menu_lem

### Lista de Problemas

O arquivo `erros,json` mostra os erros encontrados após executar o comando `rasa_nlu.evaluate.run_evaluation`.

Normalmente os erros mostrados são textos repetidos nos exemplos de diferrentes `intents`.

Caso o arquivo não seja gerado significa que não foram encontrados erros.

In [5]:
%cat errors.json

cat: errors.json: No such file or directory


### Avaliação do modelo de NLU treinado

Caso queira testar mensagens específicas mande a mensagem na célula seguinte:

In [6]:
pprint(interpreter.parse('oi'))

{
  "intent": {
    "name": "cumprimentar",
    "confidence": 0.9420649409294128
  },
  "entities": [],
  "intent_ranking": [
    {
      "name": "cumprimentar",
      "confidence": 0.9420649409294128
    },
    {
      "name": "despedir",
      "confidence": 0.21336573362350464
    },
    {
      "name": "tudo_bem",
      "confidence": 0.13765519857406616
    },
    {
      "name": "civel_propositura",
      "confidence": 0.11990172415971756
    },
    {
      "name": "menu_agendamento",
      "confidence": 0.11697175353765488
    },
    {
      "name": "civel_propositura_pessoafisica_localmoradia",
      "confidence": 0.09347838908433914
    },
    {
      "name": "civel",
      "confidence": 0.07425334304571152
    },
    {
      "name": "infancia",
      "confidence": 0.04320669174194336
    },
    {
      "name": "conhecimento",
      "confidence": 0.04159783571958542
    },
    {
      "name": "menu_lembrete",
      "confidence": 0.04120170697569847
    }
  ],
  "text": "oi"
}
